In [2]:
import os
import joblib

import pandas as pd
import numpy as np

In [20]:
foldder = './smt'
df_list = []

eval_dict = {
    'f1': [],
    'precision': [],
    'recall': [],
}

for file in os.listdir(foldder):
    element = file.replace('.csv', '').split('_')
    f1 = float(element[0]) / 10000
    prec = float(element[1]) / 10000
    rec = float(element[2]) / 10000
    eval_dict['f1'].append(f1)
    eval_dict['precision'].append(prec)
    eval_dict['recall'].append(rec)
    file_path = os.path.join(foldder, file)
    df_list.append(file_path)

In [4]:
def create_df(path_list):
    prob_df = pd.DataFrame()
    pred_df = pd.DataFrame()
    for path in path_list:
        sub_df = pd.read_csv(path)
        prob_df = pd.concat([prob_df, sub_df.iloc[:, 2]], axis=1)
        pred_df = pd.concat([pred_df, sub_df.iloc[:, 1]], axis=1)
    prob_df = prob_df.reset_index(drop=True)
    pred_df = pred_df.reset_index(drop=True)
    
    # prob_df['ID'] = sub_df['id']
    # pred_df['ID'] = sub_df['id']
    return prob_df, pred_df

In [5]:
prob_df, pred_df = create_df(df_list)

In [26]:
f1_np = np.array(eval_dict['f1'])
precision_np = np.array(eval_dict['precision'])
recall_np = np.array(eval_dict['recall'])

weight_pred = pred_df.values * precision_np

In [14]:
guess = pd.read_csv('../../testing/guess136.csv')
f_guess = guess.copy()

zero_list = [
    6722, 8831, 11999, 7555, 22570, 6898, 20029, 14120,
    14467, 4160, 12519, 8484, 9251, 18104, 18850, 15676, 16995,
    23508, 15749, 24661, 19364, 13181, 5418, 7613, 23976, 780,
    23557, 10288, 5375, 17790, 3479,2731,7752,6833,22045,23492,18305,14120,4328
]
zero_np = np.array(zero_list) - 1
for i in zero_np:
    f_guess.iloc[i, 1] = 0.5

/tmp/ipykernel_2289621/2722370535.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  f_guess.iloc[i, 1] = 0.5


In [29]:
en_prob_df = pd.DataFrame({
    'ID': guess['ID'],
    'prob': prob_df.mean(axis=1)[:25108].values,
    'pred': pred_df.mean(axis=1)[:25108].values,
    'weight_pred': weight_pred.mean(axis=1)[:25108]
})

In [42]:
en_prob_df.merge(f_guess, on='ID', how='outer').sort_values(by=['pred', '飆股'], ascending=False).reset_index(drop=True)

,ID,prob,pred,weight_pred,飆股
0,PU-10090,9.751420e-01,1.0,0.925595,1.0
1,PU-10091,9.897870e-01,1.0,0.925595,1.0
2,PU-1024,9.998955e-01,1.0,0.925595,1.0
3,PU-10251,9.637167e-01,1.0,0.925595,1.0
4,PU-10413,1.000000e+00,1.0,0.925595,1.0
...,...,...,...,...,...
25103,PU-9994,5.378237e-10,0.0,0.000000,0.0
25104,PU-9995,1.137533e-09,0.0,0.000000,0.0
25105,PU-9996,6.871566e-10,0.0,0.000000,0.0
25106,PU-9998,3.529868e-09,0.0,0.000000,0.0


In [50]:
def f1(prec, rec):
    return 2 * (prec * rec) / (prec + rec)
f1(0.772, 0.9)

0.8311004784688996